In [1]:
import pandas as pd
from random import randint

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Intel\AppData\Local\Temp\ipykernel_19244\3871533204.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
from selenium.webdriver.common.by import By
from dateutil import parser

symbols = ['PANI', 'JPFA', 'AALI', 'ULTJ', 'FAPA', 'SMAR', 'SSMS']

from time import sleep

df = pd.DataFrame()

for symbol in symbols:
    driver.get(f"https://stockbit.com/symbol/{symbol}")
    
    # Handdle: IndexError: list index out of range
    while True:
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        
        # Handle: IndexError: list index out of range
        if len(date_elements) != 0:
            break

    while True:
        # date_elements[-2] Handle: StaleElementReferenceException or ''
        # .text[:16] Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
        if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
            date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        else:
            break
    
    # value='stream-text-single' Handle: prevent Repost to be included in the comment_elements    
    comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text-single")
    
    dates = list()
    comments = list()
    
    for index, (date_element, comment_element) in enumerate(zip(date_elements, comment_elements)):
        # print(date_element.text, comment_element.text, "\n")
        
        # Handle: StaleElementReferenceException or ''
        if index == len(date_elements) - 1:
            break
        
        dates.append(date_element.text)
        comments.append(comment_element.text)
        
    print(f'symbol {symbol} '
          f'date_elements {len(date_elements)} dates {len(dates)} '
          f'comment_elements {len(comment_elements)} comments {len(comments)} ')
    
    df_c = pd.DataFrame()
    df_c['date'] = dates
    df_c['comment'] = comments
    df_c['symbol'] = symbol
    
    df = pd.concat([df, df_c])
    
    sleep(randint(5,10))

symbol PANI date_elements 1161 dates 1160 comment_elements 1181 comments 1160 
symbol JPFA date_elements 341 dates 340 comment_elements 341 comments 340 
symbol AALI date_elements 401 dates 400 comment_elements 401 comments 400 
symbol ULTJ date_elements 501 dates 500 comment_elements 521 comments 500 
symbol FAPA date_elements 21 dates 20 comment_elements 21 comments 20 
symbol SMAR date_elements 81 dates 80 comment_elements 81 comments 80 
symbol SSMS date_elements 221 dates 220 comment_elements 241 comments 220 


In [6]:
df

,date,comment,symbol
0,"19 Dec 22, 09:46",Owner $PANI,PANI
1,"18 Dec 22, 20:55 · Edited on 18 Dec 22, 21:05",$SLIS $PANI $BEBS $BUMI\n\nKenapa CBUT bisa BA...,PANI
2,"18 Dec 22, 20:55",$PANI,PANI
3,"18 Dec 22, 20:54 · Edited on 19 Dec 22, 07:15",$PANI holdernya kasihan masih harap balik ke 2...,PANI
4,"18 Dec 22, 15:12",$PANI kalau PANI mau lebih banyak di minati .....,PANI
...,...,...,...
215,"30 Sep 22, 16:41",Sawit Sumbermas (SSMS) Yakin Bisa Teruskan Kin...,SSMS
216,"30 Sep 22, 13:10",$SSMS,SSMS
217,"29 Sep 22, 14:02",$SSMS 27 Sep 22\nShareholder : Putra Borneo Ag...,SSMS
218,"29 Sep 22, 11:33",$SSMS ... kok sepiiiiii amat.... mencurigakan,SSMS


In [7]:
df.to_csv("PANI_SSMS.csv", index=False)